# Instalação das bibliotecas para a atividade

In [5]:
!pip install transformers python-docx pypdf

#Leitura dos arquivos docx e pdf, e geração dos arquivos de texto com os dados;

- Definição das funções que realizam a leitura do arquivo docx e pdf, e gera a string com os dados do arquivo;



In [6]:
from docx import Document
from pypdf import PdfReader

def extract_docx_data(doc):
    docx_str = ""
    for index, table in enumerate(doc.tables):
        rows = table.rows
        cells = rows[0].cells
        size_cells = len(cells)

        if (size_cells <= 2):
           docx_str += f"\nTable: {cells[0].text}\n"
           continue

        for row in rows:
            docx_str += str([cell.text for cell in row.cells]) + "\n"
    return docx_str

def extract_pdf_data(doc):
    pdf_str = ""
    for page in doc.pages:
        pdf_str += page.extract_text()
    return pdf_str

- Extração dos dados dos arquivos

In [7]:
docx_path = 'DICIONARIO_DE_DADOS.docx'
pdf_path = 'doencas_respiratorias_cronicas.pdf'
docx_document = Document(docx_path)
pdf_document = PdfReader(pdf_path)
docx_data = extract_docx_data(docx_document)
pdf_data = extract_pdf_data(pdf_document)

# Importação da biblioteca pipeline

In [8]:
from transformers import pipeline

# Geração dos vetores de Questing e definição dos modelo usados na atividade

> As perguntas acabaram não sendo viáveis para modelos de QA devido a natureza
dos resultados, logo possibilitaremos múltiplas respostas


In [9]:
questions_docx = ['Qual tabela representa o indicador LFCES058?',
                  'Quais campos das tabelas se repetem com maior frequência?',
                  'Quais tabelas possuem algum elemento no campo DOMINIO?']

questions_pdf = ['Quais são os meios de tratar uma rinite alergica?',
                 'Como corticoide pode ser utilizada e quais são suas contraindicações?',
                 'O que é tabagismo?']

expected_resp_docx = [
    ['TB_ESTAB_BANCO'],
    ['UNIDADE_ID', 'DATA_ATU', 'DT_ATUALIZACAO', 'USUARIO', 'CO_USUARIO', 'CHKSUM', 'STATUS', 'STATUSMOV', 'DT_ATUALIZACAO_ORIGEM', 'DT_CMTP_INICIO', 'DT_CMTP_FIM', 'NU_SEQ_PROCESSO'],
    ['FCESGEST', 'TB_ESTABELECIMENTO', 'TB_SERVICO_REFERENCIADO', 'TB_COLETA_SELETIVA_REJEITO', 'TB_SERVICO_APOIO', 'TB_ATIVIDADE_PROFISSIONAL', 'TB_CBO']
]

expected_resp_pdf = [
    ['beta-agonista', 'alérgenos', 'anti-histamínico', 'corticoide', 'broncodilatadores'],
    ['imunossupressor', 'dexametasona', 'gotas', 'injeções', 'intranasais', 'intramuscular', 'intranasal', 'perfuração', 'via oral', 'sedação', 'irritação', 'sangramento'],
    ['nicotina', 'dependência quimica à droga nicotina', 'doença crônica']
]

models = ['eraldoluis/faquad-bert-base-portuguese-cased',
          'pierreguillou/bert-base-cased-squad-v1.1-portuguese',
          'timpal0l/mdeberta-v3-base-squad2']

# Preparação dos dados para o context que serão usados nos modelos

- Extração da tabela que contém a relações de tabelas no arquivo docx

In [10]:
relations_tables = docx_data.split("\n\n")

- Extração das tabelas do arquivo docx

In [11]:
tables = ""
for item in relations_tables:
    if "Table" in item:
        tables += item + "\n\n"

# Geração das repostas

In [12]:
models_resp = []

def show_model_resp(pipe, question, context, array_resp):
    resp = pipe(question=question, context=context)
    res = resp['answer']
    score = resp['score']

    obj_resp = {'question': question, 'answer': res}
    array_resp.append(obj_resp)
    print(f'Question: {question}')
    print(f'- Answer: {repr(res)} ({score:.2f})')
    print()

for i, model in enumerate(models):
    obj = {'model_name': model, 'docx_resp': [], 'pdf_resp': []}
    models_resp.append(obj)

    pipe = pipeline('question-answering', model=model, tokenizer=model)

    print(f'Model: {model}\n')

    print(f"Document: {docx_path}\n")
    context = relations_tables[0]
    array_resp = models_resp[i]['docx_resp']
    show_model_resp(pipe, questions_docx[0], context, array_resp)
    context = tables
    for question in questions_docx[1:]:
        show_model_resp(pipe, question, context, array_resp)

    print()

    print(f"Document: {pdf_path}\n")
    context = pdf_data
    array_resp = models_resp[i]['pdf_resp']
    for question in questions_pdf:
        show_model_resp(pipe, question, context, array_resp)

    print()


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/872 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/433M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/566 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

...

tokenizer.json:   0%|          | 0.00/16.3M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/23.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/173 [00:00<?, ?B/s]

Device set to use cuda:0


Model: timpal0l/mdeberta-v3-base-squad2

Document: DICIONARIO_DE_DADOS.docx

Question: Qual tabela representa o indicador LFCES058?
- Answer: " 'TB_ESTAB_BANCO'," (0.54)

Question: Quais campos das tabelas se repetem com maior frequência?
- Answer: ' ACESSOSUSUARIO' (0.00)

Question: Quais tabelas possuem algum elemento no campo DOMINIO?
- Answer: ' NFCES003' (0.24)


Document: doencas_respiratorias_cronicas.pdf

Question: Quais são os meios de tratar uma rinite alergica?
- Answer: ' inalação nasal exclusiva' (0.50)

Question: Como corticoide pode ser utilizada e quais são suas contraindicações?
- Answer: '\n[somente uso EV]' (1.15)

Question: O que é tabagismo?
- Answer: ' um \nproblema de saúde pública' (1.23)




# Verificação de similaridade das respostas usando similaridade por cosseno

In [13]:
from sentence_transformers import SentenceTransformer, util

def find_cossine_similarity(sentence_transformer_model, expected_resp, model_resp):
    embedding_expected_resp = sentence_transformer_model.encode(expected_resp, convert_to_tensor=True)
    embedding_model_resp = sentence_transformer_model.encode(model_resp, convert_to_tensor=True)
    return util.pytorch_cos_sim(embedding_expected_resp, embedding_model_resp).item()


def show_resp(sentence_transformer_model, expected_resp, model_resp):
    cos_sim = find_cossine_similarity(sentence_transformer_model, expected_resp, model_resp)
    return (cos_sim, expected_resp)


def display_results(document_path, question_list, model_resp_list, expected_resp_list, sentence_transformer_model):
    print(f'Document: {document_path}\n')

    for j in range(len(question_list)):
        print(f'Question: {question_list[j]}')
        print(f'Model response: {repr(model_resp_list[j]["answer"])}\n')

        results = []
        for expected_resp in expected_resp_list[j]:
            results.append(show_resp(sentence_transformer_model, expected_resp, model_resp_list[j]['answer']))

        sorted_results = sorted(results, key=lambda x: x[0], reverse=True)
        for cos_sim, expected_resp in sorted_results:
            print(f'- Expected: {expected_resp} ({cos_sim*100:.4f}%)')

        print()


# Carrega um modelo pre-treinado de sentece transformer
sentence_transformer_model = SentenceTransformer('all-MiniLM-L6-v2')

for i in range(len(models)):
    print(f'Model Name: {models[i]}')

    display_results(docx_path, questions_docx, models_resp[i]['docx_resp'], expected_resp_docx, sentence_transformer_model)
    display_results(pdf_path, questions_pdf, models_resp[i]['pdf_resp'], expected_resp_pdf, sentence_transformer_model)


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Model Name: eraldoluis/faquad-bert-base-portuguese-cased
Document: DICIONARIO_DE_DADOS.docx

Question: Qual tabela representa o indicador LFCES058?
Model response: 'TB_ESTAB_BANCO'

- Expected: TB_ESTAB_BANCO (100.0000%)

Question: Quais campos das tabelas se repetem com maior frequência?
Model response: 'LFCES041'

- Expected: STATUS (29.4740%)
- Expected: DT_CMTP_INICIO (27.6058%)
- Expected: DT_CMTP_FIM (23.6120%)
- Expected: STATUSMOV (18.6908%)
- Expected: UNIDADE_ID (18.0120%)
- Expected: CO_USUARIO (16.1177%)
- Expected: CHKSUM (14.5271%)
- Expected: DATA_ATU (14.1738%)
- Expected: USUARIO (13.7313%)
- Expected: DT_ATUALIZACAO_ORIGEM (9.9767%)
- Expected: NU_SEQ_PROCESSO (9.3915%)
- Expected: DT_ATUALIZACAO (9.1406%)

Question: Quais tabelas possuem algum elemento no campo DOMINIO?
Model response: 'FCESGEST'

- Expected: FCESGEST (100.0000%)
- Expected: TB_ESTABELECIMENTO (30.0133%)
- Expected: TB_COLETA_SELETIVA_REJEITO (27.4155%)
- Expected: TB_SERVICO_APOIO (26.3638%)
- Expec

In [14]:
print("--- Model Performance Summary ---")

for i in range(len(models)):
    print(f'\nModel Name: {models[i]}')
    total_similarity = 0
    num_questions = 0

    print(f"\nDocument: {docx_path}")
    for j in range(len(questions_docx)):
        num_questions += 1
        results = []
        for expected_resp in expected_resp_docx[j]:
            results.append(find_cossine_similarity(sentence_transformer_model, expected_resp, models_resp[i]['docx_resp'][j]['answer']))
        max_similarity = max(results) if results else 0
        total_similarity += max_similarity
        print(f"  Question: {questions_docx[j]} - Max Cosine Similarity: {max_similarity:.4f}")

    print(f"\nDocument: {pdf_path}")
    for j in range(len(questions_pdf)):
        num_questions += 1
        results = []
        for expected_resp in expected_resp_pdf[j]:
            results.append(find_cossine_similarity(sentence_transformer_model, expected_resp, models_resp[i]['pdf_resp'][j]['answer']))
        max_similarity = max(results) if results else 0
        total_similarity += max_similarity
        print(f"  Question: {questions_pdf[j]} - Max Cosine Similarity: {max_similarity:.4f}")

    average_similarity = total_similarity / num_questions if num_questions > 0 else 0
    models_resp[i]['average_similarity'] = average_similarity
    print(f"\nAverage Cosine Similarity for {models[i]}: {average_similarity:.4f}")

# Find the best model
best_model = None
highest_average_similarity = -1

for model_info in models_resp:
    if model_info['average_similarity'] > highest_average_similarity:
        highest_average_similarity = model_info['average_similarity']
        best_model = model_info['model_name']

print(f"\n--- Best Model ---")
print(f"The model with the highest average cosine similarity is: {best_model} ({highest_average_similarity:.4f})")

--- Model Performance Summary ---

Model Name: eraldoluis/faquad-bert-base-portuguese-cased

Document: DICIONARIO_DE_DADOS.docx
  Question: Qual tabela representa o indicador LFCES058? - Max Cosine Similarity: 1.0000
  Question: Quais campos das tabelas se repetem com maior frequência? - Max Cosine Similarity: 0.2947
  Question: Quais tabelas possuem algum elemento no campo DOMINIO? - Max Cosine Similarity: 1.0000

Document: doencas_respiratorias_cronicas.pdf
  Question: Quais são os meios de tratar uma rinite alergica? - Max Cosine Similarity: 0.8811
  Question: Como corticoide pode ser utilizada e quais são suas contraindicações? - Max Cosine Similarity: 1.0000
  Question: O que é tabagismo? - Max Cosine Similarity: 1.0000

Average Cosine Similarity for eraldoluis/faquad-bert-base-portuguese-cased: 0.8626

Model Name: pierreguillou/bert-base-cased-squad-v1.1-portuguese

Document: DICIONARIO_DE_DADOS.docx
  Question: Qual tabela representa o indicador LFCES058? - Max Cosine Similarity